# Healing ts approaches with missing value imputation

One of the problem of predictive clustering algorithm is missing points. Depends in our algorithm modification we can have more or less missing point, but after some time we can't predict any more because we have to much of them. For dealing with this problem we can apply healing algorithms. In this work we would like to try approaches discussed in article [Long-term missing value imputation for time series data using deep neural networks](https://link.springer.com/article/10.1007/s00521-022-08165-6). 


Having read the article carefully we have identified 3 approaches:
- [imputeTS](https://github.com/SteffenMoritz/imputeTS) R package, that provide several different imputation algorithm, from witch we choose 3 for now:
    1. missing Value Imputation by Interpolation,
    2. missing Value Imputation by Kalman Smoothing,
    3. missing Value Imputation by Weighted Moving Average;
- [mtsdi](https://cran.r-project.org/web/packages/mtsdi/index.html) R package, that provide EM algorithm based method for imputation of missing values in multivariate normal time series;
- Radial basis function network (RBF) with hyper parameters optimization (HPO).
Unfortunately there are no implementation provided for RBF with HPO, so we will try best hyperparameter from article and will compare them with `inputeTS` and `mtsdi`. 


For dealing with r packages we will use [rpy2](https://pypi.org/project/rpy2/) package and for RBF we will create our [torch](https://pytorch.org/docs/stable/torch.html) implementation.

## Getting ready with inputeTS

In [2]:
import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector
from rpy2.robjects.packages import importr

In [3]:
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

<rpy2.rinterface_lib.sexp.NULLType object at 0x105da50d0> [0]

In [6]:
utils.install_packages('imputeTS')

R[write to console]: устанавливаю также зависимости ‘utf8’, ‘commonmark’, ‘xfun’, ‘stringi’, ‘xts’, ‘TTR’, ‘nlme’, ‘Matrix’, ‘farver’, ‘labeling’, ‘munsell’, ‘R6’, ‘RColorBrewer’, ‘viridisLite’, ‘fansi’, ‘pillar’, ‘pkgconfig’, ‘curl’, ‘markdown’, ‘png’, ‘jpeg’, ‘stringr’, ‘xml2’, ‘quadprog’, ‘quantmod’, ‘jsonlite’, ‘lattice’, ‘cli’, ‘glue’, ‘gtable’, ‘isoband’, ‘lifecycle’, ‘MASS’, ‘mgcv’, ‘rlang’, ‘scales’, ‘tibble’, ‘vctrs’, ‘withr’, ‘gridtext’, ‘colorspace’, ‘fracdiff’, ‘generics’, ‘lmtest’, ‘nnet’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘ggplot2’, ‘ggtext’, ‘stinepack’, ‘forecast’, ‘magrittr’, ‘Rcpp’


R[write to console]: пробую URL 'https://cloud.r-project.org/src/contrib/utf8_1.2.4.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 241081 bytes (235 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write t

x86_64-apple-darwin13.4.0-clang -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG   -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-version-min=10.9 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include   -Iutf8lite/src -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1698915189859/work=/usr/local/src/conda/r-base-4.3.2 -fdebug-prefix-map=/Users/alexglushko/miniconda3/envs/ts-horizon=/usr/local/src/conda-prefix  -c as_utf8.c -o as_utf8.o
x86_64-apple-darwin13.4.0-clang -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG   -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-version-min=10.9 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include   -Iutf8lite/src -fPIC  -marc

ld: warning: -pie being ignored. It is only used when linking a main executable
installing to /Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/library/00LOCK-utf8/00new/utf8/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (utf8)
* installing *source* package ‘commonmark’ ...
** пакет ‘commonmark’ удачно распакован, MD5 sums проверены
** using staged installation
** libs
using C compiler: ‘clang version 16.0.6’
using SDK: ‘MacOSX13.3.sdk’


rm -f cmark/cmark.o cmark/node.o cmark/iterator.o cmark/blocks.o cmark/inlines.o cmark/scanners.o cmark/utf8.o cmark/buffer.o cmark/references.o cmark/render.o cmark/man.o cmark/xml.o cmark/html.o cmark/commonmark.o cmark/latex.o cmark/houdini_href_e.o cmark/houdini_html_e.o cmark/houdini_html_u.o cmark/cmark_ctype.o cmark/arena.o cmark/linked_list.o cmark/plugin.o cmark/registry.o cmark/syntax_extension.o cmark/plaintext.o cmark/footnotes.o cmark/map.o extensions/autolink.o extensions/core-extensions.o extensions/ext_scanners.o extensions/strikethrough.o extensions/table.o extensions/tagfilter.o extensions/tasklist.o extensions.o init.o wrapper.o commonmark.dylib cmark/libstatcmark.a
x86_64-apple-darwin13.4.0-clang -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG -Icmark -I. -DCMARK_GFM_STATIC_DEFINE -DCMARK_GFM_EXTENSIONS_STATIC_DEFINE -DR_NO_REMAP -DSTRICT_R_HEADERS  -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-v

ld: warning: -pie being ignored. It is only used when linking a main executable
installing to /Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/library/00LOCK-commonmark/00new/commonmark/libs
** R
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (commonmark)
* installing *source* package ‘xfun’ ...
** пакет ‘xfun’ удачно распакован, MD5 sums проверены
** using staged installation
** libs
using C compiler: ‘clang version 16.0.6’
using SDK: ‘MacOSX13.3.sdk’


...

x86_64-apple-darwin13.4.0-clang -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG   -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-version-min=10.9 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include    -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1698915189859/work=/usr/local/src/conda/r-base-4.3.2 -fdebug-prefix-map=/Users/alexglushko/miniconda3/envs/ts-horizon=/usr/local/src/conda-prefix  -c read.c -o read.o
x86_64-apple-darwin13.4.0-clang -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG   -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-version-min=10.9 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include    -fPIC  -march=core2 -mtune=haswell -mssse3 -ft

ld: warning: -pie being ignored. It is only used when linking a main executable
installing to /Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/library/00LOCK-jpeg/00new/jpeg/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (jpeg)
* installing *source* package ‘xml2’ ...
** пакет ‘xml2’ удачно распакован, MD5 sums проверены
** using staged installation


Found pkg-config cflags and libs!
Using PKG_CFLAGS=-I/Users/alexglushko/miniconda3/envs/ts-horizon/include/libxml2 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include
Using PKG_LIBS=-L/Users/alexglushko/miniconda3/envs/ts-horizon/lib -lxml2 -L/Users/alexglushko/miniconda3/envs/ts-horizon/lib -lz -L/Users/alexglushko/miniconda3/envs/ts-horizon/lib -llzma -L/Users/alexglushko/miniconda3/envs/ts-horizon/lib -liconv -L/Users/alexglushko/miniconda3/envs/ts-horizon/lib -licui18n -licuuc -licudata


** libs
using C compiler: ‘clang version 16.0.6’
using C++ compiler: ‘clang version 16.0.6’
using SDK: ‘MacOSX13.3.sdk’


x86_64-apple-darwin13.4.0-clang++ -std=gnu++17 -I"/Users/alexglushko/miniconda3/envs/ts-horizon/lib/R/include" -DNDEBUG -I../inst/include -I/Users/alexglushko/miniconda3/envs/ts-horizon/include/libxml2 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include -DUCHAR_TYPE=wchar_t -DU_SHOW_CPLUSPLUS_API=0  -D_FORTIFY_SOURCE=2 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -mmacosx-version-min=10.9 -I/Users/alexglushko/miniconda3/envs/ts-horizon/include    -fPIC  -march=core2 -mtune=haswell -mssse3 -ftree-vectorize -fPIC -fPIE -fstack-protector-strong -O2 -pipe -stdlib=libc++ -fvisibility-inlines-hidden -fmessage-length=0 -isystem /Users/alexglushko/miniconda3/envs/ts-horizon/include -fdebug-prefix-map=/Users/runner/miniforge3/conda-bld/r-base-split_1698915189859/work=/usr/local/src/conda/r-base-4.3.2 -fdebug-prefix-map=/Users/alexglushko/miniconda3/envs/ts-horizon=/usr/local/src/conda-prefix  -c connection.cpp -o connection.o
x86_64-apple-darwin13.4.0-clang -I"/Users/alexg

In [5]:
imputeTS = importr('imputeTS')

PackageNotInstalledError: The R package "imputeTS" is not installed.